In [ ]:
import pandas as pd
import os
import datetime as dt
import numpy as np
from sklearn.impute import KNNImputer
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

# Marktstammdatenregisters

### Read data and combined


# Applied mask, grouped by Inbetriebnahmedatum and sum of values

In [ ]:
def read_and_combine_files(start, end, folder_path):
    # Liste zum Speichern der eingelesenen DataFrames
    dfs = []

    # Durchlaufe den Bereich der Nummern
    for i in range(start, end + 1, 5000):
        # Erstelle den Dateinamen basierend auf dem Nummernbereich
        file_name = f"Stromerzeuger_{i}_bis_{i + 4999}.csv" 

        # Erstelle den vollständigen Pfad zur CSV-Datei
        file_path = os.path.join(folder_path, file_name)

        # Lese das CSV in einen DataFrame ein und füge ihn zur Liste hinzu
        df = pd.read_csv(file_path, delimiter=';')
        dfs.append(df)

    # Lese die letzte Datei ein
    last_file_path = os.path.join(folder_path, 'Stromerzeuger_465001_bis_465169.csv')
    last_df = pd.read_csv(last_file_path, delimiter=';')

    # Füge die letzte Datei zum DataFrame hinzu
    dfs.append(last_df)

    # Kombiniere alle DataFrames nach dem Index
    combined_df = pd.concat(dfs, axis=0, ignore_index=True)
    return combined_df

folder_path = r'C:\Users\mohdr\Portfolio project\Marktstammdatenregister'

combined_df = read_and_combine_files(1, 464999, folder_path)

combined_df.head()

In [ ]:
#selected_bruttoleistung = combined_df.groupby('Inbetriebnahmedatum der Einheit')['Bruttoleistung der Einheit'].apply(lambda x: x.str.replace(',', '.').astype(float).sum())

#print(selected_bruttoleistung)

In [ ]:
# DataFrame mit Datum erstellen
df_combined = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end='12/31/2023', freq='1d')})

# Nur die Zeilen filtern, die den 'Betriebs-Status' 'In Betrieb' haben
mask_combined_df = (combined_df['Betriebs-Status'] == 'In Betrieb')
df_combined_filtered = combined_df.loc[mask_combined_df, :].copy()  # Kopie erstellen

# 'Inbetriebnahmedatum der Einheit' in Datumstyp konvertieren
df_combined_filtered['Inbetriebnahmedatum der Einheit'] = pd.to_datetime(df_combined_filtered['Inbetriebnahmedatum der Einheit'], format="%d.%m.%Y", errors='coerce')

# Gruppensummen für 'Bruttoleistung der Einheit' und 'Nettonennleistung der Einheit' basierend auf 'Inbetriebnahmedatum der Einheit' berechnen
grouped_sum = df_combined_filtered.groupby('Inbetriebnahmedatum der Einheit').agg({
    'Bruttoleistung der Einheit': lambda x: x.str.replace(',', '.').astype(float).sum(),
    'Nettonennleistung der Einheit': lambda x: x.str.replace(',', '.').astype(float).sum()
}).reset_index()

# Spaltennamen ändern
grouped_sum = grouped_sum.rename(columns={'Bruttoleistung der Einheit': 'Bruttoleistung', 'Nettonennleistung der Einheit': 'Nettoleistung'})

# Gruppensummen in den DataFrame df_combined einfügen
df_combined = df_combined.merge(grouped_sum, left_on='date', right_on='Inbetriebnahmedatum der Einheit', how='left').fillna(0)

# Ergebnisse anzeigen
df_combined = df_combined.drop(columns=['Inbetriebnahmedatum der Einheit'])  # Optional: Entfernen Sie die zusätzliche Spalte
print(df_combined)


# Bayern
### Tatsächliche und prognostizierte Solarenergieeinspeisung in der Regelzone von TenneT Deutschland - Bayern

In [ ]:
# data source: https://netztransparenz.tennet.eu/de/strommarkt/transparenz/transparenz-deutschland/netzkennzahlen/tatsaechliche-und-prognostizierte-solarenergieeinspeisung/bayern/
# filtered for 2023-01-01 to 2023-12-31

df_en_raw = pd.read_table('solarEnergyFeedIn_BY_2015-01-01_2023-12-31.csv',sep=';',parse_dates=['Datum'],decimal=',')

df_en = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end='12/31/2023', freq='1d')})

df_en.loc[:,'Prog_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Prognostiziert in MW'].sum()).iloc[:,1]
df_en.loc[:,'Act_in_MW'] = pd.DataFrame(df_en_raw.groupby(['Datum'], as_index=False)['Tatsaechlich in MW'].sum()).iloc[:,1]

#df = df.merge(df_en,on='date')
#df_en

In [ ]:
df_en_comb = pd.merge(df_combined,df_en, on = 'date')
df_en_comb = pd.DataFrame(df_en_comb)
print(df_en_comb)

# Stationsmessungen der Solarstrahlung

tageswerte_ST_03668, 05792, 05856, 05705 und 02290 \\
07370 und 00867 Started 2021 and 05404 Ended 2014, therefore these stations are excluded

In [ ]:
df_stations = pd.DataFrame({'date': pd.date_range(start='1/1/2015', end='31/12/2023', freq='1d')})
#'07370' Started 2021, '05404' Ended 2014 and '00867 Started 2021', therefore these stations are excluded

stationsid = ['03668', '05792', '05856', '05705', '02290', '05404']

for id in stationsid:
    stationsname = f'stationid_{id}.txt'
    
    # Construct the full path to the station file within the 'Station' folder
    station_path = os.path.abspath(os.path.join(os.getcwd(), 'Stations', stationsname))
    
    df_stat = pd.read_csv(station_path, sep=';', parse_dates=['MESS_DATUM'], usecols=['MESS_DATUM', 'FD_STRAHL', 'FG_STRAHL', 'SD_STRAHL'])
    
    mask = (df_stat['MESS_DATUM'] >= '2015-01-01') & (df_stat['MESS_DATUM'] <= '2023-12-31')
    
    df_stat.columns = ['date', f'{id}_FD_STRAHL', f'{id}_FG_STRAHL', f'{id}_SD_STRAHL']
    
    # Merge based on the 'date' column
    df_stations = df_stations.merge(df_stat.loc[mask, :], on='date')
df_stations.head()

# Final Combination DataFrame from 3 Tables

In [ ]:
df_final = df_stations.merge(df_en_comb, on = 'date')
df_final.head()
#df_final.to_csv('df_final_raw_2015_2023.csv')

In [ ]:
df = pd.read_csv('us_df_final_2015_2023.csv', index_col=0)

# Annahme: df ist dein DataFrame mit den Zeitreihen-Daten
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Angenommen, deine Daten sind bereits eingelesen
df.index = pd.to_datetime(df.index)
df.index.freq = 'D'  # Hier die entsprechende Frequenz einfügen

df['Act_in_MW'].plot(figsize=(12, 6), title='Stromproduktion')
plt.show()
